In [1]:
from pathlib import Path
from os.path import join
from google.cloud import bigquery
from google.oauth2 import service_account


path = Path(__file__).resolve()
ROOT_PATH = path.parents[2]
ENV_PATH = ROOT_PATH / 'env'
CREDENTIALS_PATH = str(ENV_PATH / 'google_gfw_credentials.json')

In [2]:

credentials = service_account.Credentials.from_service_account_file( CREDENTIALS_PATH, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)


In [3]:
table = "gfw-testing-377510.fishing_efforts.fishing_data"
query = f"""
SELECT
  CONCAT(FORMAT ("%'.2f", SUM(CASE
          WHEN flag_gfw IS NOT NULL THEN 1
        ELSE
        0
      END
        ) / COUNT(*) * 100), '%') AS num_flag_gfw,
  CONCAT(FORMAT ("%'.2f", SUM(CASE
          WHEN flag_ais IS NOT NULL THEN 1
        ELSE
        0
      END
        ) / COUNT(*) * 100), '%') AS num_flag_ais,
  CONCAT(FORMAT ("%'.2f", SUM(CASE
          WHEN flag_registry IS NOT NULL THEN 1
        ELSE
        0
      END
        ) / COUNT(*) * 100), '%') AS num_flag_registry
FROM
  {table}
  """
query_job = client.query(query)

In [4]:
query_job.to_dataframe()

,num_flag_gfw,num_flag_ais,num_flag_registry
0,99.22%,79.79%,21.59%
